# **Imports**

In [1]:
import os
import sys
import tqdm
from pathlib import Path
from tqdm.notebook import trange, tqdm
from warnings import filterwarnings
from dotenv import load_dotenv

load_dotenv()
filterwarnings("ignore")

PATH = os.getcwd()
PROJECT = str(Path(PATH).parents[0])
GOOGLE_MAPS_API = os.getenv("GOOGLE_MAPS_API")

In [2]:
import numpy as np
import pandas as pd
import ampligraph
import requests
from ampligraph.datasets import load_from_csv
import json
from urllib.parse import quote_plus

ampligraph.__version__

'1.4.0'

## **Preprocess**

In [3]:
# url = f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=Anejo&inputtype=textquery&key={GOOGLE_MAPS_API}"

# respon = requests.get(url)
# json.loads(respon.text)
# url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id=ChIJr_tCsVg1K4gREHA5C-YHJys&fields=types,rating,price_level&key={GOOGLE_MAPS_API}" #
# respon = requests.get(url)
# json.loads(respon.text)

In [4]:
def get_place_id(place):
    try:
        place = quote_plus(place)
        url = f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={place}+Toronto&inputtype=textquery&key={GOOGLE_MAPS_API}"
        response = requests.get(url)
        return json.loads(response.text)["candidates"][0]["place_id"]
    except:
        return None

def get_place_details(place_id):
    try:
        url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&key={GOOGLE_MAPS_API}" #fields=types,rating,price_level
        response = requests.get(url)
        return json.loads(response.text)
    except:
        return None

In [82]:
# file = f"{PROJECT}/data/maps/Want to go.csv"
# df = pd.read_csv(file)[["Title"]]
# df["place_id"] = df.Title.map(lambda x: get_place_id(x))
# df.to_csv(file, index=False)

# file = f"{PROJECT}/data/maps/Favorite places.csv"
# df = pd.read_csv(file)[["Title"]]
# df["place_id"] = df.Title.map(lambda x: get_place_id(x))
# df.to_csv(file, index=False)

,Title,place_id
0,Shook,ChIJ8eB4-940K4gR2K8ckRlXgmY
1,Sofia Restaurant & Bar,ChIJJWuXY6U0K4gRRc0QwSyBsN8
2,Sunnyside Beach,ChIJUZPdgcg1K4gRAqHCKWed9bM
3,Stock T.C,ChIJ10YjyKUzK4gRY_JulT4V3SM
4,Colossal Cookie Company,None


In [132]:
# file = f"{PROJECT}/data/maps/Saved Places.json"
# df = pd.read_json(file)
# df = pd.DataFrame(list(df.features))
# df["Title"] = df["properties"].apply(lambda x: x.get("Title"))
# df["place_id"] = df.Title.map(lambda x: get_place_id(x))
# file = f"{PROJECT}/data/maps/Saved places.csv"
# df[["Title", "place_id"]].to_csv(file, index=False)
# df

,geometry,properties,type,Title,place_id
0,"{'coordinates': [-79.3983552, 43.6436227], 'ty...",{'Google Maps URL': 'http://maps.google.com/?c...,Feature,Marben,ChIJ7xo6It80K4gR7DCOJQPoG5c
1,"{'coordinates': [-79.3976425, 43.702135], 'typ...",{'Google Maps URL': 'http://maps.google.com/?c...,Feature,Mariachi's Restaurant,ChIJ2azwrT4zK4gRu1wznQVfdEs
2,"{'coordinates': [-79.4015223, 43.6529421], 'ty...",{'Google Maps URL': 'http://maps.google.com/?c...,Feature,The Boat Restaurant,ChIJyfwvjcI0K4gR1knTSasoI-E
3,"{'coordinates': [-79.400137, 43.6528], 'type':...",{'Google Maps URL': 'http://maps.google.com/?c...,Feature,El Rey Mezcal Bar,ChIJC7ZkAsM0K4gRXV1uVuN0T-s
4,"{'coordinates': [-79.3921423, 43.6557344], 'ty...",{'Google Maps URL': 'http://maps.google.com/?c...,Feature,Krispy Kreme Café,ChIJD4ldGsY0K4gRAqNxz_S73jE
...,...,...,...,...,...
291,"{'coordinates': [-79.3805639, 43.6642444], 'ty...",{'Google Maps URL': 'http://maps.google.com/?c...,Feature,Crews & Tangos Restaurant & Bars,ChIJtQsiS7M0K4gRgR8DOWCnDQo
292,"{'coordinates': [-79.3876304, 43.6485905], 'ty...",{'Google Maps URL': 'http://maps.google.com/?c...,Feature,Cake Nightclub,ChIJ0ZKQwNE0K4gRoXARYglzPEk
293,"{'coordinates': [-79.4006213, 43.6582582], 'ty...",{'Google Maps URL': 'http://maps.google.com/?c...,Feature,Comfort Zone,ChIJu7jbC7U1K4gR4DKZBWOEBqM
294,"{'coordinates': [-79.3805207, 43.6426286], 'ty...",{'Google Maps URL': 'http://maps.google.com/?q...,Feature,"Maple Leaf Square, Bremner Blvd, Toronto, ON M...",ChIJaf1cydQ0K4gRWI0gKkEZETY


In [83]:
file = f"{PROJECT}/data/maps/Reviews.json"
df = pd.read_json(file)
df = pd.DataFrame(list(df.features))
df["Title"] = df["properties"].apply(lambda x: x.get("Location", {}).get("Business Name"))
df["place_id"] = df.Title.map(lambda x: get_place_id(x))
df["rating"] = df["properties"].map(lambda x: x.get("Star Rating"))
file = f"{PROJECT}/data/maps/Reviews.csv"
df[["Title", "place_id", "rating"]].to_csv(file, index=False)
df.head()

,geometry,properties,type,Title,place_id,rating
0,"{'coordinates': [-79.4312411, 43.6418312], 'ty...",{'Google Maps URL': 'https://www.google.com/ma...,Feature,Chantecler Restaurant,ChIJLUr2fqw1K4gRW4a0y6aVPsk,1
1,"{'coordinates': [-79.38154, 43.656375], 'type'...",{'Google Maps URL': 'https://www.google.com/ma...,Feature,LCBO,ChIJA8YL6MM0K4gRAuSHOYkkOEw,4
2,"{'coordinates': [-79.382693, 43.6565962], 'typ...",{'Google Maps URL': 'https://www.google.com/ma...,Feature,Atrium,ChIJ455Vqss0K4gR36Q6R4jCPsc,4
3,"{'coordinates': [-79.3847361, 43.6583305], 'ty...",{'Google Maps URL': 'https://www.google.com/ma...,Feature,Fresh City,ChIJtakM6Ro1K4gRvnzChakEf5A,5
4,"{'coordinates': [-79.298267, 43.670736], 'type...",{'Google Maps URL': 'https://www.google.com/ma...,Feature,Foodland - Toronto,ChIJUf_aWIPM1IkRTJH0Kleso5w,4


### **Create Knowledge Graphs**

In [5]:
files = ["Want to go", "Saved places", "Favorite places"]
relations = ["want_to_go", "saved", "is_favorite"]

df_master = pd.DataFrame()

for i, file in enumerate(files):
    file_path = f"{PROJECT}/data/maps/{file}.csv"
    df_tmp = pd.read_csv(file_path)
    df_tmp["relation"] = relations[i]
    print(df_tmp.shape, df_tmp.place_id.nunique())
    df_master = pd.concat([df_master, df_tmp])

print(df_master.shape, df_master.place_id.nunique())
df_master["place_details"] = df_master.place_id.map(lambda x: get_place_details(x))
df_master.head()

(413, 3) 380
(296, 3) 281
(69, 3) 68
(778, 3) 717


,Title,place_id,relation,place_details
0,Shook,ChIJ8eB4-940K4gR2K8ckRlXgmY,want_to_go,"{'html_attributions': [], 'result': {'address_..."
1,Sofia Restaurant & Bar,ChIJJWuXY6U0K4gRRc0QwSyBsN8,want_to_go,"{'html_attributions': [], 'result': {'address_..."
2,Sunnyside Beach,ChIJUZPdgcg1K4gRAqHCKWed9bM,want_to_go,"{'html_attributions': [], 'result': {'address_..."
3,Stock T.C,ChIJ10YjyKUzK4gRY_JulT4V3SM,want_to_go,"{'html_attributions': [], 'result': {'address_..."
4,Colossal Cookie Company,NaN,want_to_go,"{'html_attributions': [], 'status': 'INVALID_R..."


In [6]:
df_master.head()

,Title,place_id,relation,place_details
0,Shook,ChIJ8eB4-940K4gR2K8ckRlXgmY,want_to_go,"{'html_attributions': [], 'result': {'address_..."
1,Sofia Restaurant & Bar,ChIJJWuXY6U0K4gRRc0QwSyBsN8,want_to_go,"{'html_attributions': [], 'result': {'address_..."
2,Sunnyside Beach,ChIJUZPdgcg1K4gRAqHCKWed9bM,want_to_go,"{'html_attributions': [], 'result': {'address_..."
3,Stock T.C,ChIJ10YjyKUzK4gRY_JulT4V3SM,want_to_go,"{'html_attributions': [], 'result': {'address_..."
4,Colossal Cookie Company,NaN,want_to_go,"{'html_attributions': [], 'status': 'INVALID_R..."


In [8]:
df_master["lat"] = df_master.place_details.map(lambda x: x.get("result", {}).get("geometry", {}).get("location", {}).get("lat"))
df_master["lon"] = df_master.place_details.map(lambda x: x.get("result", {}).get("geometry", {}).get("location", {}).get("lng"))
df_master["price_level"] = df_master.place_details.map(lambda x: x.get("result", {}).get("price_level"))
df_master["rating"] = df_master.place_details.map(lambda x: x.get("result", {}).get("rating"))
df_master["types"] = df_master.place_details.map(lambda x: x.get("result", {}).get("types"))
df_master["user_ratings_total"] = df_master.place_details.map(lambda x: x.get("result", {}).get("user_ratings_total"))
df_master.head()

,Title,place_id,relation,place_details,lat,lon,price_level,rating,types,user_ratings_total
0,Shook,ChIJ8eB4-940K4gR2K8ckRlXgmY,want_to_go,"{'html_attributions': [], 'result': {'address_...",43.644040,-79.399728,NaN,4.2,"[restaurant, food, point_of_interest, clothing...",228.0
1,Sofia Restaurant & Bar,ChIJJWuXY6U0K4gRRc0QwSyBsN8,want_to_go,"{'html_attributions': [], 'result': {'address_...",43.670719,-79.391865,4.0,4.3,"[restaurant, food, point_of_interest, establis...",575.0
2,Sunnyside Beach,ChIJUZPdgcg1K4gRAqHCKWed9bM,want_to_go,"{'html_attributions': [], 'result': {'address_...",43.636766,-79.452018,NaN,4.5,"[natural_feature, establishment]",131.0
3,Stock T.C,ChIJ10YjyKUzK4gRY_JulT4V3SM,want_to_go,"{'html_attributions': [], 'result': {'address_...",43.709271,-79.399276,NaN,4.3,"[restaurant, grocery_or_supermarket, food, poi...",350.0
4,Colossal Cookie Company,NaN,want_to_go,"{'html_attributions': [], 'status': 'INVALID_R...",NaN,NaN,NaN,NaN,None,NaN


In [9]:
df_master[(df_master.lat.round(2) == 43.71) & (df_master.lon.round(2) == -79.40)]

,Title,place_id,relation,place_details,lat,lon,price_level,rating,types,user_ratings_total
3,Stock T.C,ChIJ10YjyKUzK4gRY_JulT4V3SM,want_to_go,"{'html_attributions': [], 'result': {'address_...",43.709271,-79.399276,NaN,4.3,"[restaurant, grocery_or_supermarket, food, poi...",350.0
195,Restaurant A.A,ChIJn-zqWTszK4gRvdjPgpxF4Q8,want_to_go,"{'html_attributions': [], 'result': {'address_...",43.706837,-79.397160,1.0,3.9,"[restaurant, food, point_of_interest, establis...",758.0
237,HK Sweets,ChIJRc1graszK4gRb8wrEtZk6k4,want_to_go,"{'html_attributions': [], 'result': {'address_...",43.709403,-79.398687,NaN,4.4,"[cafe, restaurant, food, point_of_interest, es...",195.0
262,La Vecchia Restaurant,ChIJa5o8oSMzK4gR5Lbbne-B_E8,want_to_go,"{'html_attributions': [], 'result': {'address_...",43.710192,-79.398861,2.0,4.4,"[restaurant, food, point_of_interest, establis...",1224.0
385,De Mello Palheta Coffee Roasters,ChIJJWWw2iMzK4gRguD50KdIGxQ,want_to_go,"{'html_attributions': [], 'result': {'address_...",43.711844,-79.399172,2.0,4.7,"[cafe, bakery, food, point_of_interest, store,...",856.0
193,Mandarin Restaurant,ChIJg482lDwzK4gRXv0WroRd75o,saved,"{'html_attributions': [], 'result': {'address_...",43.705879,-79.398329,2.0,4.2,"[restaurant, food, point_of_interest, establis...",3096.0


In [113]:
df_relations = df_master.copy()
df_relations = df_relations[df_relations.lat.notna()]
# df_relations.fillna({"price_level": "99", })
#pd.DataFrame()
X = []
for row in df_relations.itertuples():
    X.append(["Lejin", row.relation, row.Title])
    if not pd.isna(row.price_level):
        X.append([row.Title, "price_level", row.price_level])
    X.append([row.Title, "lat", round(row.lat, 2)])
    X.append([row.Title, "lon", round(row.lon,2)])
    if not pd.isna(row.rating):
        X.append([row.Title, "rating", round(float(row.rating))])
    if not pd.isna(row.user_ratings_total):
        X.append([row.Title, "user_ratings_total", float(row.user_ratings_total)])
    for i, place_type  in enumerate(row.types):
        if place_type not in (["point_of_interest", "establishment", "store"]):
            X.append([row.Title, "place_type", place_type])
print(len(X))
X[:3]

5705


[['Lejin', 'want_to_go', 'Shook'],
 ['Shook', 'lat', 43.64],
 ['Shook', 'lon', -79.4]]

In [114]:
X = np.array(X)
entities = np.unique(np.concatenate([X[:, 0], X[:, 2]]))
relations = np.unique(X[:, 1])
print(relations, entities)

['is_favorite' 'lat' 'lon' 'place_type' 'price_level' 'rating' 'saved'
 'user_ratings_total' 'want_to_go'] ['-115.17' '-118.46' '-118.5' ... 'transit_station' 'university'
 'veterinary_care']


In [115]:
from ampligraph.evaluation import train_test_split_no_unseen
from ampligraph.latent_features import ComplEx
import tensorflow as tf

X_train, X_test = train_test_split_no_unseen(X, test_size=500) 
print('Train set size: ', X_train.shape)
print('Test set size: ', X_test.shape)

Train set size:  (5205, 3)
Test set size:  (500, 3)


# **Train**

## **ComplEx**

In [116]:
model = ComplEx(batches_count=100, 
                seed=0, 
                epochs=100, 
                k=64, 
                eta=5,
                optimizer='adam', 
                optimizer_params={'lr':1e-3},
                loss='multiclass_nll', 
                regularizer='LP', 
                regularizer_params={'p':3, 'lambda':1e-5}, 
                verbose=True)

In [117]:
positives_filter = X
tf.logging.set_verbosity(tf.logging.ERROR)
model.fit(X_train, early_stopping = False)

Average ComplEx Loss:   0.241190: 100%|██████████| 100/100 [00:50<00:00,  1.99epoch/s]


In [118]:
from ampligraph.latent_features import save_model, restore_model
from ampligraph.evaluation import evaluate_performance
save_model(model, './best_model.pkl')

ranks = evaluate_performance(X_test, 
                             model=model, 
                             filter_triples=positives_filter,   # Corruption strategy filter defined above 
                             use_default_protocol=True, # corrupt subj and obj separately while evaluating
                             verbose=True)

from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score

mrr = mrr_score(ranks)
print("MRR: %.2f" % (mrr))

hits_10 = hits_at_n_score(ranks, n=10)
print("Hits@10: %.2f" % (hits_10))
hits_3 = hits_at_n_score(ranks, n=3)
print("Hits@3: %.2f" % (hits_3))
hits_1 = hits_at_n_score(ranks, n=1)
print("Hits@1: %.2f" % (hits_1))

WARNING - DeprecationWarning: use_default_protocol will be removed in future. Please use corrupt_side argument instead.


100%|██████████| 500/500 [00:02<00:00, 207.02it/s]


MRR: 0.27
Hits@10: 0.51
Hits@3: 0.31
Hits@1: 0.16


In [119]:
X_unseen = np.array([
    ['Lejin', 'is_favorite', 'Butter Baker'],
    ['Lejin', 'is_favorite', 'Plaja Tuzla'],
    ['Lejin', 'is_favorite', 'Hong Shing Chinese Restaurant'],
    ['Lejin', 'is_favorite', 'Suite 114'],
    ['Lejin', 'is_favorite', 'The Piston'],
    ['Lejin', 'is_favorite', 'Poutineville'], #Sofia Restaurant & Bar
    ['Lejin', 'is_favorite', 'Sunnyside Beach'],
    ['Lejin', 'is_favorite', 'Sofia Restaurant & Bar'],
    ['Lejin', 'is_favorite', 'Don Valley Brick Works Park'],
    ['Lejin', 'is_favorite', 'Stock T.C'],#
    ['Lejin', 'is_favorite', "Taro's Fish"],
    ['Lejin', 'is_favorite', 'PROJECT:FISH'],
])

unseen_filter = np.array(list({tuple(i) for i in np.vstack((positives_filter, X_unseen))}))
ranks_unseen = evaluate_performance(
    X_unseen, 
    model=model, 
    filter_triples=unseen_filter,   # Corruption strategy filter defined above 
    corrupt_side = 's+o',
    use_default_protocol=False, # corrupt subj and obj separately while evaluating
    verbose=True
)
scores = model.predict(X_unseen)

from scipy.special import expit
probs = expit(scores)

pd.DataFrame(list(zip([' '.join(x) for x in X_unseen], 
                      ranks_unseen, 
                      np.squeeze(scores),
                      np.squeeze(probs))), 
             columns=['statement', 'rank', 'score', 'prob']).sort_values("rank")

100%|██████████| 12/12 [00:00<00:00, 45.63it/s]


,statement,rank,score,prob
0,Lejin is_favorite Butter Baker,62,3.618303,0.973873
2,Lejin is_favorite Hong Shing Chinese Restaurant,185,2.810482,0.943240
3,Lejin is_favorite Suite 114,246,2.471545,0.922123
7,Lejin is_favorite Sofia Restaurant & Bar,375,1.959718,0.876502
11,Lejin is_favorite PROJECT:FISH,463,1.426204,0.806309
9,Lejin is_favorite Stock T.C,512,1.179418,0.764843
4,Lejin is_favorite The Piston,594,0.899568,0.710861
6,Lejin is_favorite Sunnyside Beach,816,0.558957,0.636211
5,Lejin is_favorite Poutineville,959,0.430931,0.606096
8,Lejin is_favorite Don Valley Brick Works Park,976,0.334619,0.582883


In [120]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

embeddings = model.get_embeddings(entities)
tsne = TSNE(n_components=2)
x_reduce  = tsne.fit_transform(embeddings)

In [123]:
import plotly.io as plt_io
import plotly.graph_objects as go
%matplotlib inline

def plot_2d(component1, component2, labels):
    
    fig = go.Figure(data=go.Scatter(
        x = component1,
        y = component2,
        mode='markers',
        text = labels,
        marker=dict(
            size=20,
#             color=y, #set color equal to a variable
            colorscale='Rainbow', # one of plotly colorscales     
            showscale=True,
            line_width=1
        )
    ))
    fig.update_layout(margin=dict( l=100,r=100,b=100,t=100),width=2000,height=1200)                 
    fig.layout.template = 'plotly'
    
#     fig.show()
    fig.write_html('output.html', auto_open=True)

In [124]:
plot_2d(x_reduce[:, 0], x_reduce[:, 1], entities)

## **TransE**

In [36]:
from ampligraph.latent_features import TransE

In [37]:
model = TransE(batches_count=100, 
                seed=0, 
                epochs=100, 
                k=64, 
                eta=5,
                optimizer='adam', 
                optimizer_params={'lr':1e-3},
                loss='multiclass_nll', 
                regularizer='LP', 
                regularizer_params={'p':3, 'lambda':1e-5}, 
                verbose=True)

In [38]:
positives_filter = X
tf.logging.set_verbosity(tf.logging.ERROR)
model.fit(X_train, early_stopping = False)

Average TransE Loss:   0.221610: 100%|██████████| 100/100 [00:25<00:00,  3.92epoch/s]


In [39]:
from ampligraph.latent_features import save_model, restore_model
from ampligraph.evaluation import evaluate_performance
save_model(model, './best_model.pkl')

ranks = evaluate_performance(X_test, 
                             model=model, 
                             filter_triples=positives_filter,   # Corruption strategy filter defined above 
                             use_default_protocol=True, # corrupt subj and obj separately while evaluating
                             verbose=True)

from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score

mrr = mrr_score(ranks)
print("MRR: %.2f" % (mrr))

hits_10 = hits_at_n_score(ranks, n=10)
print("Hits@10: %.2f" % (hits_10))
hits_3 = hits_at_n_score(ranks, n=3)
print("Hits@3: %.2f" % (hits_3))
hits_1 = hits_at_n_score(ranks, n=1)
print("Hits@1: %.2f" % (hits_1))

WARNING - DeprecationWarning: use_default_protocol will be removed in future. Please use corrupt_side argument instead.


100%|██████████| 500/500 [00:01<00:00, 422.09it/s]


MRR: 0.33
Hits@10: 0.50
Hits@3: 0.36
Hits@1: 0.24


In [40]:
X_unseen = np.array([
    ['Lejin', 'is_favorite', 'Butter Baker'],
    ['Lejin', 'is_favorite', 'Plaja Tuzla'],
    ['Lejin', 'is_favorite', 'Hong Shing Chinese Restaurant'],
    ['Lejin', 'is_favorite', 'Suite 114'],
    ['Lejin', 'is_favorite', 'The Piston'],
    ['Lejin', 'is_favorite', 'Poutineville'], #Sofia Restaurant & Bar
    ['Lejin', 'is_favorite', 'Sunnyside Beach'],
    ['Lejin', 'is_favorite', 'Sofia Restaurant & Bar'],
    ['Lejin', 'is_favorite', 'Don Valley Brick Works Park'],
    ['Lejin', 'is_favorite', 'Stock T.C'],#
    ['Lejin', 'is_favorite', "Taro's Fish"],
    ['Lejin', 'is_favorite', 'PROJECT:FISH'],
])

unseen_filter = np.array(list({tuple(i) for i in np.vstack((positives_filter, X_unseen))}))
ranks_unseen = evaluate_performance(
    X_unseen, 
    model=model, 
    filter_triples=unseen_filter,   # Corruption strategy filter defined above 
    corrupt_side = 's+o',
    use_default_protocol=False, # corrupt subj and obj separately while evaluating
    verbose=True
)
scores = model.predict(X_unseen)

from scipy.special import expit
probs = expit(scores)

pd.DataFrame(list(zip([' '.join(x) for x in X_unseen], 
                      ranks_unseen, 
                      np.squeeze(scores),
                      np.squeeze(probs))), 
             columns=['statement', 'rank', 'score', 'prob']).sort_values("rank")

100%|██████████| 12/12 [00:00<00:00, 64.12it/s]


,statement,rank,score,prob
2,Lejin is_favorite Hong Shing Chinese Restaurant,4,-11.339640,1.189192e-05
10,Lejin is_favorite Taro's Fish,275,-13.176220,1.895132e-06
0,Lejin is_favorite Butter Baker,328,-13.358027,1.580090e-06
11,Lejin is_favorite PROJECT:FISH,357,-13.493431,1.379992e-06
5,Lejin is_favorite Poutineville,387,-13.594708,1.247075e-06
9,Lejin is_favorite Stock T.C,405,-13.669506,1.157200e-06
4,Lejin is_favorite The Piston,439,-13.798346,1.017312e-06
3,Lejin is_favorite Suite 114,560,-14.388867,5.636300e-07
7,Lejin is_favorite Sofia Restaurant & Bar,622,-15.020218,2.997796e-07
6,Lejin is_favorite Sunnyside Beach,629,-15.067114,2.860457e-07


## **TransE**

In [62]:
from ampligraph.latent_features import ConvKB

In [68]:
model = ConvKB(batches_count=24, seed=0, epochs=100, k=64, eta=5,
embedding_model_params={'num_filters': 32, 'filter_sizes': [1], 'dropout': 0.1},
optimizer='adam', optimizer_params={'lr': 1e-3},regularizer='LP', regularizer_params={'p':3, 'lambda':1e-5}, 
loss='multiclass_nll', loss_params={}, verbose=True)

In [69]:
positives_filter = X
tf.logging.set_verbosity(tf.logging.ERROR)
model.fit(X_train, early_stopping = False)

Average ConvKB Loss:   0.226102: 100%|██████████| 100/100 [02:05<00:00,  1.26s/epoch]


In [70]:
from ampligraph.latent_features import save_model, restore_model
from ampligraph.evaluation import evaluate_performance
save_model(model, './best_model.pkl')

ranks = evaluate_performance(X_test, 
                             model=model, 
                             filter_triples=positives_filter,   # Corruption strategy filter defined above 
                             use_default_protocol=True, # corrupt subj and obj separately while evaluating
                             verbose=True)

from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score

mrr = mrr_score(ranks)
print("MRR: %.2f" % (mrr))

hits_10 = hits_at_n_score(ranks, n=10)
print("Hits@10: %.2f" % (hits_10))
hits_3 = hits_at_n_score(ranks, n=3)
print("Hits@3: %.2f" % (hits_3))
hits_1 = hits_at_n_score(ranks, n=1)
print("Hits@1: %.2f" % (hits_1))

WARNING - DeprecationWarning: use_default_protocol will be removed in future. Please use corrupt_side argument instead.


100%|██████████| 500/500 [00:21<00:00, 22.81it/s]


MRR: 0.30
Hits@10: 0.45
Hits@3: 0.32
Hits@1: 0.22


In [71]:
X_unseen = np.array([
    ['Lejin', 'is_favorite', 'Butter Baker'],
    ['Lejin', 'is_favorite', 'Plaja Tuzla'],
    ['Lejin', 'is_favorite', 'Hong Shing Chinese Restaurant'],
    ['Lejin', 'is_favorite', 'Suite 114'],
    ['Lejin', 'is_favorite', 'The Piston'],
    ['Lejin', 'is_favorite', 'Poutineville'], #Sofia Restaurant & Bar
    ['Lejin', 'is_favorite', 'Sunnyside Beach'],
    ['Lejin', 'is_favorite', 'Sofia Restaurant & Bar'],
    ['Lejin', 'is_favorite', 'Don Valley Brick Works Park'],
    ['Lejin', 'is_favorite', 'Stock T.C'],#
    ['Lejin', 'is_favorite', "Taro's Fish"],
    ['Lejin', 'is_favorite', 'PROJECT:FISH'],
])

unseen_filter = np.array(list({tuple(i) for i in np.vstack((positives_filter, X_unseen))}))
ranks_unseen = evaluate_performance(
    X_unseen, 
    model=model, 
    filter_triples=unseen_filter,   # Corruption strategy filter defined above 
    corrupt_side = 's+o',
    use_default_protocol=False, # corrupt subj and obj separately while evaluating
    verbose=True
)
scores = model.predict(X_unseen)

from scipy.special import expit
probs = expit(scores)

pd.DataFrame(list(zip([' '.join(x) for x in X_unseen], 
                      ranks_unseen, 
                      np.squeeze(scores),
                      np.squeeze(probs))), 
             columns=['statement', 'rank', 'score', 'prob']).sort_values("rank")

100%|██████████| 12/12 [00:00<00:00, 17.06it/s]


,statement,rank,score,prob
7,Lejin is_favorite Sofia Restaurant & Bar,19,-1.686764,0.156202
2,Lejin is_favorite Hong Shing Chinese Restaurant,40,-2.203081,0.099474
5,Lejin is_favorite Poutineville,110,-2.734070,0.060993
1,Lejin is_favorite Plaja Tuzla,227,-3.295910,0.035712
9,Lejin is_favorite Stock T.C,251,-3.375922,0.033056
0,Lejin is_favorite Butter Baker,285,-3.511495,0.028987
3,Lejin is_favorite Suite 114,335,-3.698061,0.024173
11,Lejin is_favorite PROJECT:FISH,380,-3.905765,0.019729
8,Lejin is_favorite Don Valley Brick Works Park,510,-4.623731,0.009721
10,Lejin is_favorite Taro's Fish,536,-4.765077,0.008450
